<a href="https://colab.research.google.com/github/alepenaa94/TP1_Real_or_Not/blob/master/TP1_Real_or_Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Features

En este notebook la idea es realizar todo el procesamiento del set de datos, limpieza en los campos de ser necesario y sección para la generacion de los features.

## Ideas de cosas a realizar

--> Generar un N-grama <br>
--> mejorar los stopwords <br>
--> poder darle mas caracter a un tweet segun su texto <br>
--> TF-IDF <br>
--> Embeddings <br>

In [1]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('../Data/train.csv', encoding='latin-1',dtype={'id': np.uint16,'target': np.bool})
test_df = pd.read_csv('../Data/test.csv', encoding='latin-1',dtype={'id': np.uint16})

In [3]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,True
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,True
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,True
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",True
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,True


In [4]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
tweets = pd.concat([train_df,test_df])
tweets = tweets[['id','keyword','location','text','target']]

## Limpieza de la información

En base a lo aprendido en el TP1, limpiamos la información de algunos campos del dataframe.

In [6]:
tweets['keyword'] = tweets['keyword'].str.replace('%20',' ')
tweets['keyword'].fillna('None',inplace=True)
tweets['location'].fillna('Unknown',inplace=True)

## Creación de features

Volvemos a generar las columnas utilizadas para el TP1 ya que podrían resultar features útiles para el modelo de predicción.

###### Cantidad de palabras en el tweet

In [7]:
tweets['cantidad_de_palabras'] = tweets['text'].str.count(' ') + 1

###### Longitud del tweet

In [8]:
tweets['longitud_del_tweet'] = tweets['text'].str.len()

###### Cuartiles de longitud

In [9]:
tweets['longitud_del_tweet'].describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
longitud_del_tweet,10876.0,101.790916,34.141486,5.0,78.0,108.0,134.0,169.0


In [10]:
tweets.loc[tweets['longitud_del_tweet'] < 78.0,'longitud_categ'] = "0 a 25"
tweets.loc[tweets['longitud_del_tweet'] >= 78.0,'longitud_categ'] = "25 a 50"
tweets.loc[tweets['longitud_del_tweet'] >= 108.0,'longitud_categ'] = "50 a 75"
tweets.loc[tweets['longitud_del_tweet'] >= 134.0,'longitud_categ'] = "75 a 100"

Usamos el método de Smothing. (El código está basado en el obtenido de la siguiente fuente: https://gist.github.com/marnixkoops/e68815d30474786e2b293682ed7cdb01)

In [11]:
def smoothing(df, column, target, weight=100):
    mean = df[target].mean()
    agg = df.groupby(column)[target].agg(['count', 'mean'])

    dic = {}
    
    for i in df[column].unique():
        dic[i] = (agg.loc[i]['count'] * agg.loc[i]['mean'] + weight * mean) / (agg.loc[i]['count'] + weight)
        
    return dic

In [12]:
df = tweets[:len(train_df)]
df['target'] = df['target'].astype('bool')

In [13]:
long_categ_encoding_dic = smoothing(df,'longitud_categ','target')

In [14]:
tweets['longitud_categ'] = tweets['longitud_categ'].map(long_categ_encoding_dic)

###### Realiza menciones

In [15]:
tweets['tiene_menciones'] = tweets['text'].str.contains('@').astype('uint8')

###### Tweet expresivo

In [16]:
tweets['es_expresivo'] = (tweets['text'].str.contains('\!\!') | tweets['text'].str.contains('\?\?')).astype('uint8')

###### Cantidad de hashtags

In [17]:
tweets['cantidad_de_hashtags'] = tweets['text'].str.count('#')

###### Tiene links

In [18]:
tweets['tiene_links'] = tweets['text'].str.contains('http').astype('uint8')

###### Ubicaciones con tweet único

In [19]:
def location_unico(location):
    ubicaciones_unicas = tweets['location'].value_counts()[tweets['location'].value_counts() == 1].index
    return (location in ubicaciones_unicas)

In [20]:
tweets['location_unico'] = tweets['location'].map(location_unico).astype('uint8')

###### Encoding del campo keyword

Como la columna keyword es de tipo categórico, vamos a buscar una forma de codificarla.

In [21]:
keywords_encoding_dic = smoothing(df,'keyword','target')

In [22]:
tweets['keyword_encoded'] = tweets['keyword'].map(keywords_encoding_dic)

###### TF-IDF

In [23]:
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#Vamos a limpiar un poco el tweet.
pattern_exclude = '(one|dont|cant|would|im|people|go|make|time|love|amp|get|house|update|talk'+\
                  '|want|today|know|say|us|day|crush|see|back|think|look|rigth|remember|car|shes'+\
                  '|thing|let|still|lol|much|thank|take|way|youre|road|another|really|save|hows'+\
                  '|play|even|theres|everyone|feel|year|work|check|two|great|ing|like|sink|href|hr|hs'+\
                  '|every|build|youtuve|video|n|home|body|bag|photo|stay|game|start|gt|fuck|help'+\
                  '|best|well|california|end|live|e|rt|wreck|plan|full|may|ies|u|could|many|last'+\
                  '|find|service|leave|collapse|world|war|destroy|wound|break|right|hear|school)+'

def filter_words(tweet):
    tweet = re.sub(r'(\b[\w]+:\/\/[\w -\?&;#~=\.\/@]+[\w\/])', ' ', tweet)
    tweet = re.sub(r'\'', '', tweet)
    return re.sub(r'[www.]*[A-z]+.(com|gov|edu|net|mil|org|io|int)+', ' ', tweet)

def text_to_blob(tweet):
    tweet_blob = TextBlob(str(tweet))
    return ' '.join(tweet_blob.words)


def normalization(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            word_aux = word.lower().strip()
            if re.match(pattern_exclude,
                        word_aux):
                continue
            normalized_text = lem.lemmatize(word_aux,'v')
            normalized_tweet.append(normalized_text)
        return normalized_tweet

def clean(tweet):
    tweet_list = [word for word in (text_to_blob(filter_words(tweet))).split()]
    clean_tokens = [tkn for tkn in tweet_list if re.match(r'[A-z]+', tkn)]
    clean_s = ' '.join(clean_tokens)
    l_aux = normalization(clean_s.split())
    return ' '.join([word for word in l_aux if word not in stopwords.words('english')])

tweets['clean_text'] = tweets['text'].apply(clean)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FF633NG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FF633NG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FF633NG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel

docs = tweets[:len(train_df)]['clean_text'].to_list()

tfidf_vectorizer=TfidfVectorizer(analyzer='word', ngram_range=(1,3),sublinear_tf=True, min_df=5, norm='l2',max_features=500, encoding='latin-1', stop_words='english')

tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)

In [25]:
def similarity_feature(doc):
    #Calculo la similitud entre el doc y lo calculado para todo el set train.
    query=TfidfVectorizer(vocabulary=tfidf_vectorizer.vocabulary_)
    query=query.fit_transform([doc])
    s=pd.Series(linear_kernel(query,tfidf_vectorizer_vectors)[0])
    return s.sum()
    
tweets['tfidf_score']= tweets['clean_text'].apply(similarity_feature)

In [26]:
tweets['tfidf_score'].value_counts()

0.000000      987
57.567868      59
77.532286      58
106.542617     53
78.021807      45
             ... 
64.367695       1
39.233181       1
54.232346       1
26.891820       1
63.649156       1
Name: tfidf_score, Length: 5691, dtype: int64

In [27]:
del tweets['clean_text']

###### Embeddings

In [29]:
train_features = tweets[:len(train_df)].set_index('id').iloc[:,3:]
test_features = tweets[len(train_df):].set_index('id').iloc[:,4:]

Guardamos los dataframes de train y test en archivos .csv para usarlos en el Notebook de Algoritmos.

In [30]:
train_features.to_csv('../Data/train_features.csv')
test_features.to_csv('../Data/test_features.csv')